In [1]:
from appgeopy import *
from my_packages import *

#### Notes

Here is the script to produce the input for GTWR model calibration

Please run `20250921_D_Prepare_GTWR_Input_AGAIN.ipynb` after this

#### Note: 2025/10/23

- Prepare grid point (regression points) of DIFF. DISPLACEMENT to perform GWR prediction
- Only need the `backtransform_disp_df` to convert them to grid point

In [73]:
backtransform_disp_df = pd.read_pickle(
    "PostKriging_Backtransform_Data/BACKTRANSFORMED_CORRECTED_Monthly_DISPLACEMENT_CRFP_saveqgis_Oct2025_grid_grid.xz"
)
backtransform_disp_df = backtransform_disp_df.reset_index(drop=False)
# Pre-filter columns once
date_cols = backtransform_disp_df.columns[3:]
date_cols_filtered = [
    col
    for col in date_cols
    if (int(col[1:5]) - 2016) * 12 + int(col[5:7]) - 4 <= 67
]

# Process all rows at once
result_list = []

for select_pointkey in tqdm(backtransform_disp_df.index[:]):
    row = backtransform_disp_df.loc[select_pointkey]

    # Create base info once
    disp_info = {
        backtransform_disp_df.columns[0]: row.iloc[0],
        backtransform_disp_df.columns[1]: row.iloc[1],
        backtransform_disp_df.columns[2]: row.iloc[2],
    }

    # Build records directly
    for col in date_cols_filtered:
        monthly = (int(col[1:5]) - 2016) * 12 + int(col[5:7]) - 4
        result_list.append(
            {
                **disp_info,
                "datetime_str": col,
                "DIFFDISP": row[col],
                "monthly": monthly,
            }
        )

# Single DataFrame construction
combine_df = pd.DataFrame(result_list).sort_values(by="monthly")

  0%|          | 0/53092 [00:00<?, ?it/s]

In [74]:
combine_df.to_feather(
    r"D:\1000_SCRIPTS\003_Project002\20250917_GTWR002\3F_TestRun_6_GWR\diffdisp_gridpnt.feather"
)

In [76]:
combine_df

,PointKey,X_TWD97,Y_TWD97,datetime_str,DIFFDISP,monthly
0,X168848119Y2600592857,168848.119363,2.600593e+06,N20160501,-5.760000,1
2585798,X182448119Y2643992857,182448.119363,2.643993e+06,N20160501,-7.420000,1
2585865,X182648119Y2643992857,182648.119363,2.643993e+06,N20160501,-5.460000,1
2585932,X182848119Y2643992857,182848.119363,2.643993e+06,N20160501,-3.910000,1
264985,X184848119Y2611392857,184848.119363,2.611393e+06,N20160501,-16.010000,1
...,...,...,...,...,...,...
2336356,X198048119Y2639392857,198048.119363,2.639393e+06,N20211101,-4.034677,67
2336289,X197848119Y2639392857,197848.119363,2.639393e+06,N20211101,-4.348017,67
2336222,X197648119Y2639392857,197648.119363,2.639393e+06,N20211101,-3.883235,67
2336892,X199648119Y2639392857,199648.119363,2.639393e+06,N20211101,-1.942517,67


#### 2025/10/04

I need to extract the data after 2021 for regression points.

In [ ]:
combined_input_df = pd.DataFrame(data=None, index=None)

select_station = available_stations[-3]
for select_station in tqdm(available_stations):

    mlcw_cumpdisp_byStation = cumdisp_mlcw_df.query("STATION==@select_station")

    backtransform_cumdisp_byStation = backtransform_cumdisp_df.loc[
        select_station, cumdisp_columns
    ]

    dates = pd.to_datetime(
        [col[1:] for col in backtransform_cumdisp_byStation.index]
    )
    backtransform_cumdisp_byStation.index = dates

    mlcw_cumpdisp_byStation["CUMDISP"] = mlcw_cumpdisp_byStation.index.map(
        backtransform_cumdisp_byStation
    )

    mlcw_cumpdisp_byStation = mlcw_cumpdisp_byStation.loc[
        "2016-4":
    ]  #### just change this line

    mlcw_cumpdisp_byStation = mlcw_cumpdisp_byStation.fillna(value=0)

    combined_input_df = pd.concat([combined_input_df, mlcw_cumpdisp_byStation])

combined_input_df = combined_input_df.reset_index(drop=False)
combined_input_df = combined_input_df.sort_values(
    by=["time", "STATION"]
).reset_index(drop=True)

In [ ]:
show(combined_input_df)

In [ ]:
today_string = datetime.today().strftime("%Y%m%d")
combined_input_df.to_pickle(
    f"{today_string}_GTWR_InputData_MLCW_InSAR_regpoints.xz"
)